# Imports & Data Loading

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

import tensorflow.keras as keras 
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

W0619 20:20:50.535978 4505654720 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [3]:
raw_df = pd.read_csv('data/Political-media-DFE.csv',encoding='latin')
raw_df.head().T

,0,1,2,3,4
_unit_id,766192484,766192485,766192486,766192487,766192488
_golden,False,False,False,False,False
_unit_state,finalized,finalized,finalized,finalized,finalized
_trusted_judgments,1,1,1,1,1
_last_judgment_at,8/4/15 21:17,8/4/15 21:20,8/4/15 21:14,8/4/15 21:08,8/4/15 21:26
audience,national,national,national,national,national
audience:confidence,1,1,1,1,1
bias,partisan,partisan,neutral,neutral,partisan
bias:confidence,1,1,1,1,1
message,policy,attack,support,policy,policy


# Data Cleaning & Initial EDA

In [4]:
df = raw_df[['bias','message','embed','label','source','text']]
df.head().T

,0,1,2,3,4
bias,partisan,partisan,neutral,neutral,partisan
message,policy,attack,support,policy,policy
embed,"<blockquote class=""twitter-tweet"" width=""450"">...","<blockquote class=""twitter-tweet"" width=""450"">...","<blockquote class=""twitter-tweet"" width=""450"">...","<blockquote class=""twitter-tweet"" width=""450"">...","<blockquote class=""twitter-tweet"" width=""450"">..."
label,From: Trey Radel (Representative from Florida),From: Mitch McConnell (Senator from Kentucky),From: Kurt Schrader (Representative from Oregon),From: Michael Crapo (Senator from Idaho),From: Mark Udall (Senator from Colorado)
source,twitter,twitter,twitter,twitter,twitter
text,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...,VIDEO - #Obamacare: Full of Higher Costs and ...,Please join me today in remembering our fallen...,RT @SenatorLeahy: 1st step toward Senate debat...,.@amazon delivery #drones show need to update ...


In [5]:
df['bias'].value_counts()

neutral     3689
partisan    1311
Name: bias, dtype: int64

In [6]:
df['message'].value_counts()

policy          1411
personal        1170
support          921
information      647
media            277
attack           172
constituency     152
mobilization     129
other            121
Name: message, dtype: int64

In [7]:
#congressmen_2015 = pd.DataFrame(df['label'].unique())

In [8]:
#type(congressmen_2015)

In [9]:
#congressmen_2015.to_csv('congressmen_2015.csv')

In [10]:
def remove_punctuations(text):
    '''Removes punctuation from strings'''
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [11]:
df['text'] = df.loc[:,'text'].apply(remove_punctuations)
df['label'] = df['label'].str.replace('From: ','')
df['purpose_and_bias'] = df['message'] + '_' + df['bias']
df['text'] = df['text'].str.lower()

/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [12]:
df.head()

,bias,message,embed,label,source,text,purpose_and_bias
0,partisan,policy,"<blockquote class=""twitter-tweet"" width=""450"">...",Trey Radel (Representative from Florida),twitter,rt nowthisnews rep trey radel r fl slams obama...,policy_partisan
1,partisan,attack,"<blockquote class=""twitter-tweet"" width=""450"">...",Mitch McConnell (Senator from Kentucky),twitter,video obamacare full of higher costs and bro...,attack_partisan
2,neutral,support,"<blockquote class=""twitter-tweet"" width=""450"">...",Kurt Schrader (Representative from Oregon),twitter,please join me today in remembering our fallen...,support_neutral
3,neutral,policy,"<blockquote class=""twitter-tweet"" width=""450"">...",Michael Crapo (Senator from Idaho),twitter,rt senatorleahy 1st step toward senate debate ...,policy_neutral
4,partisan,policy,"<blockquote class=""twitter-tweet"" width=""450"">...",Mark Udall (Senator from Colorado),twitter,amazon delivery drones show need to update law...,policy_partisan


In [13]:
df['purpose_and_bias'].value_counts()

personal_neutral         1073
support_neutral           751
policy_partisan           706
policy_neutral            705
information_neutral       565
media_neutral             247
support_partisan          170
attack_partisan           160
constituency_neutral      129
other_neutral             117
personal_partisan          97
mobilization_neutral       90
information_partisan       82
mobilization_partisan      39
media_partisan             30
constituency_partisan      23
attack_neutral             12
other_partisan              4
Name: purpose_and_bias, dtype: int64

In [14]:
congressmen_df = pd.read_csv('congressmen_2015.csv')
congressmen_df.head()

,First,Last,congressman,affiliation
0,Gregorio,Sablan,Gregorio Sablan (Representative from NA),d
1,Robert,Aderholt,Robert Aderholt (Representative from Alabama),r
2,Lamar,Alexander,Lamar Alexander (Senator from Tennessee),r
3,Justin,Amash,Justin Amash (Representative from Michigan),r
4,Mark,Amodei,Mark Amodei (Representative from Nevada),r


In [15]:
df = df.merge(congressmen_df, how='left',left_on='label',right_on='congressman')

In [16]:
df.loc[df.bias == 'partisan', 'target'] = df['affiliation']
df.loc[df.bias == 'neutral', 'target'] = df['bias']

In [17]:
df.head()

,bias,message,embed,label,source,text,purpose_and_bias,First,Last,congressman,affiliation,target
0,partisan,policy,"<blockquote class=""twitter-tweet"" width=""450"">...",Trey Radel (Representative from Florida),twitter,rt nowthisnews rep trey radel r fl slams obama...,policy_partisan,Trey,Radel,Trey Radel (Representative from Florida),r,r
1,partisan,attack,"<blockquote class=""twitter-tweet"" width=""450"">...",Mitch McConnell (Senator from Kentucky),twitter,video obamacare full of higher costs and bro...,attack_partisan,Mitch,McConnell,Mitch McConnell (Senator from Kentucky),r,r
2,neutral,support,"<blockquote class=""twitter-tweet"" width=""450"">...",Kurt Schrader (Representative from Oregon),twitter,please join me today in remembering our fallen...,support_neutral,Kurt,Schrader,Kurt Schrader (Representative from Oregon),d,neutral
3,neutral,policy,"<blockquote class=""twitter-tweet"" width=""450"">...",Michael Crapo (Senator from Idaho),twitter,rt senatorleahy 1st step toward senate debate ...,policy_neutral,Michael,Crapo,Michael Crapo (Senator from Idaho),r,neutral
4,partisan,policy,"<blockquote class=""twitter-tweet"" width=""450"">...",Mark Udall (Senator from Colorado),twitter,amazon delivery drones show need to update law...,policy_partisan,Mark,Udall,Mark Udall (Senator from Colorado),d,d


In [18]:
df['target'].value_counts()

neutral    3689
r           791
d           490
i            17
Name: target, dtype: int64

In [19]:
len(df[df['target'].isnull()])

13

In [20]:
df.dropna(axis=0,inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4929 entries, 0 to 4999
Data columns (total 12 columns):
bias                4929 non-null object
message             4929 non-null object
embed               4929 non-null object
label               4929 non-null object
source              4929 non-null object
text                4929 non-null object
purpose_and_bias    4929 non-null object
First               4929 non-null object
Last                4929 non-null object
congressman         4929 non-null object
affiliation         4929 non-null object
target              4929 non-null object
dtypes: object(12)
memory usage: 500.6+ KB


## Inputs (X): Tweets
## Target (Y): Bias

# TRANSFER LEARNING

In [22]:
#download the model to local so it can be used again and again
#!mkdir module
#!mkdir module/module_elmo2

In [23]:
# Download the module, and uncompress it to the destination folder. 
#!curl -L "https://tfhub.dev/google/elmo/2?tf-hub-format=compressed" | tar -zxvC module/module_elmo2

This module exposes 4 trainable scalar weights for layer aggregation. The output dictionary contains :

- word_emb: the character-based word representations with shape [batch_size, max_length, 512].
- lstm_outputs1: the first LSTM hidden state with shape [batch_size, max_length, 1024].
- lstm_outputs2: the second LSTM hidden state with shape [batch_size, max_length, 1024].
- elmo: the weighted sum of the 3 layers, where the weights are trainable. This tensor has shape [batch_size, max_length, 1024]
- default: a fixed mean-pooling of all contextualized word representations with shape [batch_size, 1024].

# Passing Sentence Strings

In [24]:
elmo = hub.Module("module/module_elmo2/", trainable=False)
embeddings = elmo(
    ["the cat is on the mat", "what are you doing in evening"],
    signature="default",
    as_dict=True)["elmo"]

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embeddings)

Instructions for updating:
Colocations handled automatically by placer.


W0619 20:20:52.301557 4505654720 deprecation.py:323] From /anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0619 20:20:53.280848 4505654720 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [25]:
print(len(message_embeddings))
print(len(message_embeddings[0]))
print(len(message_embeddings[0][0]))

2
6
1024


# Passing Word Strings (as Tokenized Sentences)

In [26]:
elmo = hub.Module("module/module_elmo2/", trainable=False)

tokens_input = [["the", "cat", "is", "on", "the", "mat"],
["what", "are", "you", "doing", "in", "evening"]]

tokens_length = [6, 5]

embeddings = elmo(inputs={
    "tokens": tokens_input,
    "sequence_len": tokens_length
    },
    signature="tokens",
    as_dict=True)["elmo"]


with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embeddings)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0619 20:20:58.105333 4505654720 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [27]:
print(len(message_embeddings))
print(len(message_embeddings[0]))
print(len(message_embeddings[0][0]))

2
6
1024


# If we want to use for multiple inputs

In [28]:
def embed_elmo2(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

embed_fn = embed_elmo2('module/module_elmo2')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0619 20:21:02.157072 4505654720 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Graph was finalized.


I0619 20:21:02.529984 4505654720 monitored_session.py:222] Graph was finalized.


INFO:tensorflow:Running local_init_op.


I0619 20:21:03.068873 4505654720 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0619 20:21:03.155012 4505654720 session_manager.py:493] Done running local_init_op.


In [29]:
embed_fn(["i am sambit","i am alex"]).shape

(2, 1024)

# ELMO Embedding in a Simple Neural Network Classifier

In [30]:
def replace_contraction(text):
    contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'can not'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text
def replace_links(text, filler=' '):
        text = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*',
                      filler, text).strip()
        return text
def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

In [31]:
import re
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = replace_contraction(text)
    text = replace_links(text, "link")
    text = remove_numbers(text)
    text = re.sub(r'[,!@#$%^&*)(|/><";:.?\'\\}{]',"",text)
    text = text.lower()
    return text

In [32]:
X = np.array(df['text'].apply(cleanText))
y = np.array(df['target'])
y[0:10]

array(['r', 'r', 'neutral', 'neutral', 'd', 'd', 'neutral', 'neutral',
       'neutral', 'd'], dtype=object)

In [33]:
le = LabelEncoder()
y = le.fit_transform(y)

In [34]:
y[0:10]

array([3, 3, 2, 2, 0, 0, 2, 2, 2, 0])

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.20)

In [36]:
y_train.shape

(3943,)

In [37]:
embed = hub.Module("module/module_elmo2", trainable=False)
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [38]:
# def build_model(): 
#     input_text = Input(shape=(1,), dtype="string")
#     embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
#     dense = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(embedding)
#     pred = Dense(4, activation='softmax')(dense)
#     model = Model(inputs=[input_text], outputs=pred)
#     model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#     return model

In [39]:
from keras.engine.input_layer import InputLayer
from keras.models import Sequential
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(InputLayer(input_shape=(1,), dtype="string"))
model.add(Lambda(ELMoEmbedding, output_shape=(1024, ))) #
model.add(Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(Dense(50, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(Dense(4, activation='softmax'))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0619 20:21:07.113894 4505654720 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [40]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'],)
early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-05, patience=0,verbose=1, mode='auto')

callbacks_list = [early_stop]

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(X_train, y_train, 
                        epochs=5, 
                        batch_size=50, 
                        validation_data = (X_test, y_test),
                        verbose=2,
                        callbacks=callbacks_list)
    model.save_weights('./model_elmo_weights.h5')


In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Teest Loss')
plt.title('Training and Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Spare Cross Entropy Loss')
plt.legend()

plt.show()